In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import xgboost as xgb
%matplotlib inline

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [8]:
DATA_DIR = '/home/jeff/Downloads/data/statoil/'
train = pd.read_json(os.path.join(DATA_DIR, 'train.json'))
train['inc_angle'] = pd.to_numeric(train['inc_angle'],errors='coerce')
train.fillna(method='pad', inplace=True)
train.shape

(1604, 5)

In [1]:
# stat collection from https://www.kaggle.com/muonneutrino/exploration-transforming-images-in-python
def get_stats(train,label=1):
    train['max'+str(label)] = [np.max(np.array(x)) for x in train['band_'+str(label)] ]
    train['maxpos'+str(label)] = [np.argmax(np.array(x)) for x in train['band_'+str(label)] ]
    train['min'+str(label)] = [np.min(np.array(x)) for x in train['band_'+str(label)] ]
    train['minpos'+str(label)] = [np.argmin(np.array(x)) for x in train['band_'+str(label)] ]
    train['med'+str(label)] = [np.median(np.array(x)) for x in train['band_'+str(label)] ]
    train['std'+str(label)] = [np.std(np.array(x)) for x in train['band_'+str(label)] ]
    train['mean'+str(label)] = [np.mean(np.array(x)) for x in train['band_'+str(label)] ]
    train['p25_'+str(label)] = [np.sort(np.array(x))[int(0.25*75*75)] for x in train['band_'+str(label)] ]
    train['p75_'+str(label)] = [np.sort(np.array(x))[int(0.75*75*75)] for x in train['band_'+str(label)] ]
    train['mid50_'+str(label)] = train['p75_'+str(label)]-train['p25_'+str(label)]

    return train
train = get_stats(train,1)
train = get_stats(train,2)

NameError: name 'train' is not defined

In [2]:
train.head()

NameError: name 'train' is not defined

In [3]:
used_features = ['inc_angle', 'max1', 'min1', 'med1', 'std1', 'mean1', 'p25_1', 'p75_1',
    'mid50_1', 'max2', 'min2', 'med2', 'std2', 'mean2', 'p25_2', 'p75_2']
dtrain = xgb.DMatrix(train[used_features].values, train['is_iceberg'].values)
model = xgb.train({ 'objective': 'binary:logistic' }, dtrain)

NameError: name 'xgb' is not defined

In [6]:
test = pd.read_json(os.path.join(DATA_DIR, 'test.json'))

In [7]:
test = get_stats(test, 1)
test = get_stats(test, 2)

In [20]:
dtest = xgb.DMatrix(test[used_features].values)
preds = model.predict(dtest)

In [21]:
 with open(os.path.join(DATA_DIR, 'predictions.csv'), 'w') as pred_file:
        pred_file.write('id,is_iceberg\n')
        for i,pred in enumerate(preds):
            pred_file.write('{},{}\n'.format(test['id'][i], pred))